### Install Ollama and Langchain

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip install langchain_community

### Start Ollama server as a background process and pull llama3 image.

In [ ]:
import subprocess
process = subprocess.Popen(["ollama", "serve"])

In [ ]:
!ollama list

In [ ]:
!ollama pull llama3

### Expose the port

In [ ]:
from google.colab.output import eval_js
notebook_url = eval_js("google.colab.kernel.proxyPort(11434)")

In [ ]:
!export OLLAMA_HOST=notebook_url

In [ ]:
!pip install gradio

###Imports

In [1]:
from langchain_community.llms import Ollama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import gradio as gr

/home/donny/Documents/STLA/teammates/teammates_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


###Prompt Template

In [2]:
# Define your DSA-related prompt template
dsa_template = """
You are an expert in Data Structures and Algorithms. Only answer the question if it is related to the following topics:
- Data Structures (arrays, stacks, queues, linked lists, trees, heaps, graphs, etc.)
- Algorithms (sorting, searching, dynamic programming, greedy algorithms, graph algorithms, etc.)

If the question is related to these topics, provide a detailed explanation. If not, respond with: 'This is not a DSA-related question.'

{history}
Student: {input}
LLM:
"""

# Create the prompt template
prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=dsa_template
)

###Conversation Memory and Chain Setup

In [3]:
# Create a memory object which will store the conversation history.
memory = ConversationBufferMemory()

# Create a chain with this memory object and the model object created earlier.
chain = ConversationChain(
    llm = Ollama(model="llama3"),
    memory=memory,
    prompt=prompt
)

/tmp/ipykernel_198218/2851351641.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipykernel_198218/2851351641.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3"),
/tmp/ipykernel_198218/2851351641.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(


###Yes/No Response Handler

In [4]:
# Function to handle Yes/No questions
def handle_yes_no(response):
    if "yes" in response.lower():
        return "Great! What do you want to explore next?"
    elif "no" in response.lower():
        return "Understood. Would you like to ask something else? if not type 'new question' to start new topic"
    return "Thanks for your input! What’s next?"

###Process Input Function

In [5]:
def process_input(input_text):
    # Check if the user wants to reset the conversation
    if "new question" in input_text.lower():
        memory.clear()  # Reset memory
        return "Conversation memory has been reset. How can I assist you further?"

    # If the user types 'exit', 'bye', or 'quit', end the conversation and erase memory
    if input_text.lower() in ["exit", "bye", "quit"]:
        memory.clear()  # Erase memory
        return "Ending the conversation. Goodbye!"

    # Check if the input is 'yes' or 'no'
    if input_text.strip().lower() in ["yes", "no"]:
        # Handle yes/no responses without involving the LLM
        response = handle_yes_no(input_text)
        # Save the exchange to memory for context
        memory.save_context({"input": input_text}, {"output": response})
        return response

    # Generate response using the updated chain
    response = chain.predict(input=input_text)
    return response

###Interect chatbot using Python

In [6]:
def chat_with_llm():
    while True:
        input_text = input("You: ")
        response = process_input(input_text)

        if response == "Ending the conversation. Goodbye!":
            print(response)
            break

        print(f"LLM: {response}")

###Chat with chatbot using Python

In [7]:
# %%timeit

user_input = ("whats my name again")
response=chain.invoke(user_input)
print(response['response'])

This is not a DSA-related question.


In [8]:
chat_with_llm()

You:  how does this work?


LLM: This is not a DSA-related question.


You:  what is sorting?


LLM: A DSA-related question!

Sorting refers to the process of arranging elements in a collection, such as an array or list, in a specific order. The most common types of sorting algorithms are:

1. **Quicksort**: A divide-and-conquer algorithm that uses a pivot element to partition the data into two subarrays.
2. **Merge sort**: A divide-and-conquer algorithm that combines smaller sorted subarrays to produce a larger sorted array.
3. **Bubble sort**: An iterative algorithm that repeatedly iterates through the array, comparing adjacent elements and swapping them if they are in the wrong order.
4. **Selection sort**: An iterative algorithm that repeatedly selects the smallest (or largest) element from the unsorted portion of the array and moves it to the beginning (or end) of the sorted portion.

Sorting algorithms can be classified into two main categories:

1. **Comparison-based sorting**: Algorithms that compare elements to determine their order, such as quicksort and merge sort.
2. 

KeyboardInterrupt: Interrupted by user

###Gradio Integration (Interect with GPU)

In [9]:
def chatbot_response(input_text):
    return process_input(input_text)

# Gradio interface for the chatbot
iface = gr.Interface(
    fn=chatbot_response,
    inputs="text",
    outputs="text",
    title="DSA Chatbot",
    description="Ask about Data Structures and Algorithms."
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7866

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/10/18 10:26:53 [W] [service.go:132] login to server failed: dial tcp: lookup gradio-live.com: i/o timeout


**Abstract**

This project aims to create an interactive chatbot that is specific to Data Structures and Algorithms (DSA). The chatbot will help users to understand complex DSA concepts better by providing accurate, context-aware explanations while filtering out non-DSA-related queries. The chatbot uses custom prompt templates and conversation memory to maintain relevance and coherence in interactions. A DSA-specific filter is implemented through a custom prompt to ensure that the chatbot responds only to relevant topics. LangChain is used to manage the language model while Gradio is used to create a user-friendly interface. Test results have shown that the chatbot can effectively maintain conversational context, filter out unrelated queries, and provide users with personalized responses.

**Methodology**

This methodology is to develop and test the chatbot interaction in Python before it was laid into Gradio for a better user interface experience.

**1. Setup Chatbot**

To accomplish this task, we started by incorporating the LangChain library into our code to build a DSA-related chatbot. Using an instance of the Ollama model from langchain_community.llms, a chain was made for conversation. It was built on the same principles, using a customized PromptTemplate to make the chatbot talk about DSA topics only. Such a PromptTemplate made the talk agent present responses focused solely on DSA-related topics like arrays, stacks, queues, trees, and algorithms such as sorting, searching, and dynamic programming. In any other case, if there was any non-DSA question, by default, it would respond with "This is not a DSA-related question."

**2. Memory Management**

To save the context of conversations, a ConversationBufferMemory object was created. It stored the history of conversations that were in context and coherent within a session. This would be an important process to make the chatbot answer follow-up questions regarding what was previously discussed.

**3. Testing via Python**

So far, we have interacted with the chatbot through Python in a command-line environment before embedding it into a web interface like Gradio. First, it designed a function, process_input, which handled user input and checked whether the conversation was to be reset or quit-specific commands like "new question" or "exit," respectively-and handled some simple yes/no responses without having to resort to the language model. The interaction with the chatbot was tested by running a loop such that a user may input questions directly to the Python script from the terminal. For this reason, we could see the quality of the responses and refine the logic of this. An example usage test was: chat_with_llm().
This is going to simulate a user conversation by continuously receiving inputs, processing them with process_input, and generating responses from the object ConversationChain.

**4. Handling Simple Responses**

For efficiency, there was also a separate function developed to handle simple yes/ no questions. The function basically skipped the language model and returned pre-defined answers in accordance with the input of the user. For instance, if the user typed "yes," then this bot would respond with, "Great! What do you want to explore next?"

**5. Results Validation**

We ran several tests in Python to make sure that the chatbot was identifying DSA-related queries and also handled non-DSA questions. These tests have shown the process input function is able to reset the conversation memory, and stop conversations when needed.


**Results**

**1. Interaction Testing using Python**

At the earlier stages of the testing with the chatbot interaction, there were positive results. For instance when a user queried a question outside from the topic of Data Structures and Algorithm (DSA), the chatbot answered with the pre-configured messages. Hence important findings from this stage will be addressed below.

The first finding would be the response accuracy, the chatbot can  correctly identify among queries that were DSA topic related or not. When identified, the chatbot provides thorough explanations on related topics such as such sorting algorithms, graph traversal and tree architectures.

Moreover memory management, the conversation buffer memory performed as expected which enables the chatbot to save conversation history. With this feature, the chatbot is able to respond to follow up questions with the similar context as before.

Lastly is the reaction time, the python chatbot was efficient to facilitate seamless communication and provide the user to close real time interaction.

**2. Identified Improvements**

During the project development, prior to the usage of Gradio there were many issues found that needed to be improved to further enhance by testing. They are as such the handling of yes or no questions, the handle_yes_no function is able to deal with these questions. However, the testing indicated that other edge situations should be improved because when users use phrases like “yup” or “nah” the handling or yes or no may be flawed.

Furthermore, the feature is the chatbot memory reset functionality. The chatbot is able to start a new reset when the user typed “new question” However because users may use different terms such as “start over”, the reset mechanism needs to be improved to be able to work with such prompts.

Lastly, is the termination of chat, using the commands such as “exit”, it ends the conversation as expected from our implemented function but the team found that it would be better to enhance the user experience by providing an optional confirmation question before the session ends.

**3. Transition to Gradio**

Based on our testing and finding, the team decided to incorporate Gradio, a web based interface. This required the team to integrate Gradio’s text input/output functions with the existing process_input function. The transition to Gradio was smooth and the following improvements are as follows.

Firstly, it is observed to have smooth integration. During testing, the chatbot retained its functionality with the Gradio interface. Hence, every essential function and features such as memory retention and precise DSA related response were working smoothly.

In addition to the improvement of user-friendly interfaces, Gradio allowed users to communicate with the chatbot in real time easily. The chatbot had stayed consistent and delivered timely responses.

Lastly, the chatbot's scalability has improved. By allowing the chatbot to be shared with others for extensive testing, it helps the team to confirm and check that the chatbot was indeed ready for deployment.

**4. Feedback from Gradio’s user**

Early users of the Gradio interface offered insightful feedback. The majority of the users found that the chatbot was useful in providing DSA related questions and further suggested that the team can further expand the chatbot to handle more complex DSA topics. Users seemed to enjoy the chatbot capacity to efficiently answer the yes or no questions and being able to restart conversations with the chatbot.

**Conclusion**

The project is efficient in creating a specialized chatbot that efficiently filters out irrelevant context out of the DSA topic, while offering in depth assistance on the DSA queries from users. The integration of the Gradio interface and the Langchain language model management, the chatbot provides users such as educators and students to improve the comprehension of DSA topics which is the main goal of the project.


**Key Achievements:**

Contextual Awareness:  The chatbot preserves conversational context, allowing for smooth, multi-turn interactions that closely resemble natural human conservations.

Focused Expertise: Through a carefully crafted prompt, the chatbot only responds to  DSA topics, making it an effective educational tool.

User Engagement: The inclusion of tailored responses to yes/no inputs enriches the interaction, encouraging users to explore the subject more thoroughly.

**Limitations:**

Reliance on LLM Performance: The accuracy of the chatbot depends on the language model's ability to interpret and generate responses, which may occasionally result in inaccurate responses.

Ambiguity Handling: Following context resets, the chatbot occasionally encounters difficulty with ambiguous queries, emphasizing the need for improved mechanisms to prompt clarification when necessary.

**Future Work:**

Improved Ambiguity Resolution: Develop methods for the chatbot to request clarification when faced with ambiguous queries, particularly after context resets, to ensure better accuracy.

Expanded Knowledge Base: Broaden the chatbot’s scope by incorporating additional subjects or more advanced DSA concepts to increase its educational value.

Adaptive Learning Features: Introduce personalization, allowing the chatbot to tailor explanations based on the user's prior knowledge or learning pace.

Enhanced Language Models: Explore the integration of more advanced or specialized language models to improve response accuracy and reliability.

**Final Thoughts:**

This project underscores the potential of AI-driven chatbots in education, demonstrating how interactive chatbots can support learners in understanding difficult concepts. The success of this project underscores the current capabilities of AI technologies in enhancing learning experiences. Through ongoing advancements and innovations in AI, interactive chatbots could become integral components of education, offering personalized and round-the-clock support to learners worldwide. These chatbots have the potential to address challenges such as language barriers, limited access to quality education and shortage of qualified teachers.
